# else代码块
这里else指的是非if中使用的else，而是在for/while/try语句中使用的else，其含义其实更接近于‘then’：  
1.for：仅当for循环正常结束时（无breake强行终止）才执行else中的内容。
2.while：仅当while正常结束时（循环条件为假而退出而无breake强行终止）才执行else中的内容。  
3.try：无异常抛出时运行else中的内容。  
#### else块意味着上述情况正常执行完毕后才执行的内容，当有类似需求时就能节省不必要的if判断与临时变量代码。
## 举例
比如需要某句或几句代码执行无误后再执行某些代码，使用try/else能清楚的表明防御的代码以及与后续代码的因果关系。

In [ ]:
# 明确说明的仅防御dangerous_call()，无误时才能执行after_call()，如果将两者都放在try中会不够清晰。
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()

# EAFP/LBYL风格
## EAFP
python中try/except不仅用于处理错误，还常常用于控制流程。在EAFP编程风格中，一般先假定存在有效的对象（鸭子类型），如果假定不成立则给出明确错误信息，或者利用try/except执行备选方案（也可用来给出明确错误信息）。这种编程风格充分利用了python协议的优势，两个对象即便没有继承或同类型关系也能实现相同的功能。
## LBYL
LBYL风格则倾向于严格的类型检查或其他条件检查，java中使用较多，由于其严格的类型关系（形参有明确的类型声明等）。这种风格会使用较多的if/else来提前检验前置条件。  
## 推荐
使用EAFP风格的代码较为明确清晰，没有较多if/else语句对程序逻辑的干扰。
# 上下文管理器和with块
上下文管理器对象目的是管理with语句，就像迭代器对象是为了管理for语句（两种情况下python解释器会调用相关协议方法得到对象）。with语句本身的目的是为了简化try/finally模式。
## 上下文管理器协议
上下文管理器协议，需要实现\__enter__和\__exit__方法。__执行with后的表达式时得到上下文管理器对象__，该对象的\__enter__方法用于封装在进入with块时的准备逻辑，同时提供一个对象给as后面的引用（as是可选的），该引用常常作为工具对象使用。\__exit__方法用于无论以何种方式退出with块都会被执行的逻辑。

In [1]:
class LookingClass:
    
    def __enter__(self):
        import sys
        # 保存默认标准写方法引用
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    # 包装器模式
    def reverse_write(self, text):
        # 使用original_write写入反向文本
        self.original_write(text[::-1])
        
    # with块中出现异常时会传给上下文管理器，exc_type代表异常类，exc_value代表异常对象，traceback对象。
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        # 恢复
        sys.stdout.write = self.original_write
        # 仅仅解决除0异常
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            # 如果with块中出现异常，返回True告诉python解释器异常被解决，否则会向上冒泡异常。
            return True

In [3]:
with LookingClass() as what:
    print('Alice')
    print(what)
print(what)

ecilA
YKCOWREBBAJ
JABBERWOCKY


In [2]:
# 上面调用等价于
manager = LookingClass()
what = manager.__enter__()
print('Alice')
print(what)
manager.__exit__(None, None, None)
print(what)

ecilA
YKCOWREBBAJ
JABBERWOCKY


# contextlib中实用的上下文管理器
主要介绍@contextmanager，__该装饰器可用于创建上下文管理器对象，将生成器函数（生成器对象工厂）包装为上下文管理器类__。该类的\__enter__方法会使用next迭代生成器函数产生的生成器对象，并返回yield表达式对应的值。该类的\__exit__方法继续迭代生成器对象，完成剩余执行逻辑代码，当存在异常时（形参exc_type不为None）会调用生成器对象的的throw方法，使异常在生成器函数定义体中的yield行被抛出（协程？）。

In [7]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    # 包装器模式
    def reverse_write(text):
        # 使用original_write写入反向文本
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    # 当with块中出现异常而退出时，在yield处捕获该异常
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        # 由于无法得知with块是否以抛出异常结束，因此相当于__exit__的部分只能写在finally中
        sys.stdout.write = original_write
        if msg:
            print(msg)
        # 如果未解决异常，可以继续抛出，否则python解释器认为异常已被处理（不像__exit__方法，默认没处理异常）。
        else:
            raise sys.exc_info()[1]
    
# 这里使用looking_glass()产生上下文管理器对象
with looking_glass() as what:
    print('Alice')
    print(what)
print(what)

ecilA
YKCOWREBBAJ
JABBERWOCKY


感觉还是使用上下文管理器类更加清晰，使用生成器函数代替反而看上去十分笨拙

# 杂谈
with块相当于一段执行代码的前后执行逻辑封装器。